## **Chapter V. Task**

We will continue our training with a problem from Kaggle.com. 
In this chapter, we will implement all the validation schemes, some hyperparameter tuning methods, and feature selection methods described above. Measure quality metrics on training and test samples. Will detect overfitted models and regularize them. And dive deeper with native model estimation and comparison.

### 1. **Answer the questions from the introduction**

#### a. What is leave-one-out? Provide limitations and strengths.  

Leave-One-Out кросс валидация - это частный случай k-fold Cross Validation, где k = n. Данные размерности 10 делятся на обучающую и тестовую выборки по принципу 9 обучающей и 1 тестовая на каждой итерации для всех данных. При этом,каждый набор данных __ровно один раз__ становится тестовым (см. пример кода.). Другими словами, для **_n_** образцов мы имеем **_n-1_** тренировочных данных и **1** тестовый набор.
Достоинства:
 * максимальное использование данных для обучения
 * не зависит от случайного разбиения
 * эффективен только на маленьких наборах данных

Недостатки:
 * высокая стоимость вычислительных операций для больших выборок данных - O(n)
 * высокая дисперсия оценки
 * может недооценивать ошибку на новых данных
 * имеет нестабильную оценку

```Python
>>> from sklearn.model_selection import LeaveOneOut

>>> X = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
>>> loo = LeaveOneOut()
>>> for train, test in loo.split(X):
...     print("%s %s" % (train, test))

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]
```

Источник: https://scikit-learn.org/stable/modules/cross_validation.html#leave-one-out

#### b. How do Grid Search, Randomized Grid Search, and Bayesian optimization work?  

In [1]:
A = {1, 2, 3, 4, 5}
B = {6, 7, 8, 9}
C = {10, 11, 12}

def cartesian_product(*sets):
    """
    Декартово произведение множеств
    """
    result = [[]]
    for pool in sets:
        result = [x + [y] for x in result for y in pool]
    return [tuple(item) for item in result]

decart = cartesian_product(A, B, C)

> Этапы работы поиска наилучших гиперпараметров через Grid Search:
 1. **Определение пространства внешних гиперпараметров**. На первом этапе необходимо обозначить какие гиперпараметры мы ищем. Параметр должен содержать множество значений, по которым мы будем проводить поиск. Параметров может быть несколько. Несколько параметров собираются в единый словарь, где ключи — имена гиперпараметров, а значения — списки возможных значений для каждого параметра.
 2. **Создание сетки гиперпараметров**. Создается декартово произведение всех списков значений, получая полный набор всех возможных комбинаций гиперпараметров.
 3. **Проход по списку гиперпараметров**. Алгоритм GridSearch последовательно проходит по каждой комбинации гиперпараметров, обучается и оценивается с помощью k-fold CV, вычисляется средняя оценка гиперпараметров по всем фолдам для заданной метрики.
 4. **Сохранение и возвращение результата**. По окончании подбора параметров выбирается комбинация с наилучшей средней оценкой. GridSearch возвращает: лучшие гиперпараметры `best_params_`; лучшие оценки `best_score_`; полную информацию по всем комбинациям `cv_results_`.

> Randomized Grid Search:
 * Принцип работы Randomized Grid Search схож с работой Grid Search, но не создает полную сетку всех комбинаций гиперпараметров. Вместо этого поиск происходит случайным образом. Еще одно отличие - это наличие итераций. На каждой итерации `n_iter` случайным образом генерируется комбинация гиперпараметров, происходит обучение модели и оценка её качества с помощью k-fold CV на всех фолдах. Случайный выбор позволяет ускорить процесс подбора гиперпараметров. В таком случае, гиперпараметры следует задавать не только дискретным образом, но и в диапазоне и\или распределении, например: `stats.uniform(0.1, 10)`, здесь будет создано распределение случайных чисел от 0.1 до 10 + 0.1. Случайное число из данного распределения будет получено внутри GridSearch.

> Bayesian optimization


#### c. Explain classification of feature selection methods. Explain how Pearson and Chi2 work. Explain how Lasso works. Explain what permutation significance is. Become familiar with SHAP.  

### **2. Introduction — do all the preprocessing from the previous lesson**

In [2]:
# 2.Imports
import pandas as pd
import numpy as np
import re
from math import ceil

#### a. Read all the data.

In [3]:
# 2.a
df = pd.concat(
    [
        pd.read_json("../datasets/train.json", convert_dates=["created"]),
        # pd.read_json("../datasets/test.json", convert_dates=["created"])
    ]
).reset_index(drop=True)

#### b. Create features: 'Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed', 'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding', 'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck', 'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony', 'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace'.  

In [4]:
# 2.b
pattern = re.compile(r"[\[\]\'\"\s]")
remove_sym = (lambda x: list([re.sub(pattern, '', elem) for elem in x]))
df["features"] = df["features"].apply(remove_sym)

target = ["price"]
rooms = ["bedrooms", "bathrooms"]
features = ['Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed',
            'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding',
            'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck',
            'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony',
            'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace',]
for feature in features:
    df[feature] = (
        df["features"]
        .apply(lambda x: 1 if feature in x else 0)
    )

### 3. **Implement the next methods:**

#### a. Split data into 2 parts randomly with parameter test_size (ratio from 0 to 1), return training and test samples.

In [5]:
# 3.a
def my_train_test_split(X, y, test_size=0.25, random_state=None, shuffle=True):
    random = np.random.RandomState(random_state)
    if np.array_equal(X.index.to_numpy(), y.index.to_numpy()) is False:
        raise ValueError("Indices of 'X' and 'y' must be equal")

    n = len(X)
    if isinstance(test_size, int):
        if test_size <= 0 or test_size >= n:
            raise ValueError("'test_size' must be between 1 and len(X)-1")
    elif isinstance(test_size, float):
        if test_size <= 0.0 or test_size >= 1.0:
            raise ValueError("'test_size' must be between 0.0 and 1.0")
        test_size = ceil(n * test_size)
    else:
        raise ValueError("'test_size' must be float or int")

    idx = X.index.to_numpy()
    if shuffle:
        random = np.random.RandomState(random_state)
        idx = random.permutation(idx)

    train_idxs = idx[test_size:]
    test_idxs = idx[:test_size]

    return (pd.concat([X.loc[train_idxs], y.loc[train_idxs]], axis=1),
            pd.concat([X.loc[test_idxs], y.loc[test_idxs]], axis=1))


train, test = my_train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.2
)
train_idx, test_idx = train.index, test.index
print(
    "The sum of the samples is identical: " +
    f"{train.shape[0] + test.shape[0] == len(df)}",
    "\nSample sizes: " +
    f"train{train.shape}, test{test.shape}",
    "\nThere are no intersections:", all([train_idx.intersection(test_idx).empty,
                                          test_idx.intersection(train_idx).empty])
)

The sum of the samples is identical: True 
Sample sizes: train(39481, 35), test(9871, 35) 
There are no intersections: True


#### b. Randomly split data into 3 parts with parameters validation_size and test_size, return train, validation and test samples.

In [6]:
# 3.b
def my_train_validation_test_split(X,
                                   y,
                                   validation_size=0.25,
                                   test_size=0.25,
                                   random_state=None,
                                   shuffle=True):
    random = np.random.RandomState(random_state)
    if np.array_equal(X.index.to_numpy(), y.index.to_numpy()) is False:
        raise ValueError("Indices of 'X' and 'y' must be equal")

    n = len(X)
    if isinstance(test_size, int) and isinstance(validation_size, int):
        if test_size <= 0 or test_size >= n:
            raise ValueError("'test_size' must be between 1 and len(X)-1")
        if validation_size <= 0 or validation_size >= n:
            raise ValueError("'validation_size' must be between 1 and len(X)-1")
    elif isinstance(test_size, float) and isinstance(validation_size, float):
        if test_size <= 0.0 or test_size >= 1.0:
            raise ValueError("'test_size' must be between 0.0 and 1.0")
        if validation_size <= 0.0 or validation_size >= 1.0:
            raise ValueError("'validation_size' must be between 0.0 and 1.0")
        if test_size + validation_size >= 1:
            raise ValueError("test_size + validation_size must be < 1")
        test_size = ceil(n * test_size)
        validation_size = ceil(n * validation_size)
    else:
        raise ValueError("'test_size'/'validation_size' must be float or int")
    if test_size + validation_size >= n:
        raise ValueError("test_size + validation_size must be < len(X)")

    idx = X.index.to_numpy()
    if shuffle:
        random = np.random.RandomState(random_state)
        idx = random.permutation(idx)
    train_idxs = idx[test_size + validation_size:]
    test_idxs = idx[:test_size]
    validation_idxs = idx[test_size:test_size + validation_size]

    return (pd.concat([X.loc[train_idxs], y.loc[train_idxs]], axis=1),
            pd.concat([X.loc[validation_idxs], y.loc[validation_idxs]], axis=1),
            pd.concat([X.loc[test_idxs], y.loc[test_idxs]], axis=1))


train, validation, test = my_train_validation_test_split(
    df.drop(columns=target),
    df[target],
    validation_size=0.2,
    test_size=0.2,
)

train_idx, validation_idx, test_idx = train.index, validation.index, test.index
print(
    "The sum of the samples is identical: " +
    f"{train.shape[0]+validation.shape[0]+test.shape[0] == len(df)}",
    "\nSample sizes: "
    f"train{train.shape}, validation{validation.shape}, test{test.shape}",
    "\nThere are no intersections:", all([train_idx.intersection(validation_idx).empty,
                                          train_idx.intersection(test_idx).empty,
                                          validation_idx.intersection(test_idx).empty,
                                          test_idx.intersection(train_idx).empty])
)

The sum of the samples is identical: True 
Sample sizes: train(29610, 35), validation(9871, 35), test(9871, 35) 
There are no intersections: True


#### c. Split data into 2 parts with parameter date_split, return train and test samples split by date_split param.

In [7]:
# 3.c
def my_date_train_test_split(X, y, date_split=None, random_state=None, shuffle=True):
    if np.array_equal(X.index.to_numpy(), y.index.to_numpy()) is False:
        raise ValueError("Indices of 'X' and 'y' must be equal")
    if date_split is None:
        raise ValueError("'date_split' must be provided")

    dt_cols = X.select_dtypes(include=["datetime"]).columns
    if dt_cols.empty:
        raise ValueError("X does not contain a timestamp column")

    date_split = pd.to_datetime(date_split)
    if not (date_split.date() == X[dt_cols[0]].dt.date).any():
        raise ValueError(
            f"'{date_split}' is not contained in the column '{dt_cols[0]}'"
        )
    X = X.sort_values(by=dt_cols[0])
    y = y.loc[X.index]
    train_idx = X[X[dt_cols[0]] < date_split].index.to_numpy()
    test_idx = X[X[dt_cols[0]] >= date_split].index.to_numpy()

    if shuffle:
        random = np.random.RandomState(random_state)
        train_idx = random.permutation(train_idx)
        test_idx = random.permutation(test_idx)

    return (pd.concat([X.loc[train_idx], y.loc[train_idx]], axis=1),
            pd.concat([X.loc[test_idx], y.loc[test_idx]], axis=1))


train, test = my_date_train_test_split(
    df.drop(columns=target),
    df[target],
    date_split="2016-06-16",
)
train_idx, test_idx = train.index, test.index
print(
    "The sum of the samples is identical: " +
    f"{train.shape[0] + test.shape[0] == len(df)}",
    "\nSample sizes: " +
    f"train{train.shape}, test{test.shape}",
    "\nThere are no intersections:", all([train_idx.intersection(test_idx).empty,
                                          test_idx.intersection(train_idx).empty])
)
display(
    train['created'].min(),
    train['created'].max(),
    test['created'].min(),
    test['created'].max(),
)

The sum of the samples is identical: True 
Sample sizes: train(41544, 35), test(7808, 35) 
There are no intersections: True


Timestamp('2016-04-01 22:12:41')

Timestamp('2016-06-15 23:53:14')

Timestamp('2016-06-16 00:45:59')

Timestamp('2016-06-29 21:41:47')

#### d. Split data into 3 parts with parameters validation_date and test_date, return train, validation and test samples split by input params.

In [8]:
# 3.d
def my_date_train_validation_test_split(X,
                                        y,
                                        validation_date=None,
                                        test_date=None,
                                        random_state=None,
                                        shuffle=True):
    if np.array_equal(X.index.to_numpy(), y.index.to_numpy()) is False:
        raise ValueError("Indices of 'X' and 'y' must be equal")
    if test_date is None:
        raise ValueError("'test_date' must be provided")
    if validation_date is None:
        raise ValueError("'validation_date' must be provided")

    dt_cols = X.select_dtypes(include=["datetime"]).columns
    if dt_cols.empty:
        raise ValueError("X does not contain a timestamp column")

    test_date = pd.to_datetime(test_date)
    if not (test_date.date() == X[dt_cols[0]].dt.date).any():
        raise ValueError(
            f"'{test_date}' is not contained in the column '{dt_cols[0]}'"
        )

    validation_date = pd.to_datetime(validation_date)
    if not (validation_date.date() == X[dt_cols[0]].dt.date).any():
        raise ValueError(
            f"'{validation_date}' is not contained in the column '{dt_cols[0]}'"
        )
    if validation_date == test_date:
        raise ValueError(
            "'validation_date' and 'test_date' must be different"
        )
    if validation_date > test_date:
        raise ValueError(
            "'validation_date' must be less than 'test_date'"
        )
    train_idx = X[X[dt_cols[0]] < validation_date].index.to_numpy()
    valid_idx = X[(X[dt_cols[0]] >= validation_date) &
                  ((X[dt_cols[0]] < test_date))].index.to_numpy()
    test_idx = X[X[dt_cols[0]] >= test_date].index.to_numpy()
    if shuffle:
        random = np.random.RandomState(random_state)
        train_idx = random.permutation(train_idx)
        valid_idx = random.permutation(valid_idx)
        test_idx = random.permutation(test_idx)


    return (pd.concat([X.loc[train_idx], y.loc[train_idx]], axis=1),
            pd.concat([X.loc[valid_idx], y.loc[valid_idx]], axis=1),
            pd.concat([X.loc[test_idx], y.loc[test_idx]], axis=1))


train, validation, test = my_date_train_validation_test_split(
    df.drop(columns=target),
    df[target],
    validation_date="2016-05-01",
    test_date="2016-06-01",
    # random_state=21,
)
train_idx, test_idx = train.index, test.index
print(
    "The sum of the samples is identical: " +
    f"{train.shape[0] + validation.shape[0] + test.shape[0] == len(df)}",
    "\nSample sizes: " +
    f"train{train.shape}, validation{validation.shape}, test{test.shape}",
    "\nThere are no intersections:", all([train_idx.intersection(test_idx).empty,
                                          test_idx.intersection(train_idx).empty])
)
display(
    train['created'].min(),
    train['created'].max(),
    validation['created'].min(),
    validation['created'].max(),
    test['created'].min(),
    test['created'].max(),
)

The sum of the samples is identical: True 
Sample sizes: train(16411, 35), validation(15797, 35), test(17144, 35) 
There are no intersections: True


Timestamp('2016-04-01 22:12:41')

Timestamp('2016-04-30 19:21:03')

Timestamp('2016-05-01 22:36:52')

Timestamp('2016-05-31 23:10:48')

Timestamp('2016-06-01 01:10:37')

Timestamp('2016-06-29 21:41:47')

#### e. Make split procedure determenistic. What does it mean?

In [9]:
"""
    Каждая реализация my_train_test_split сопровождена детерминацией, однако,
    в функциях со стратегией 'Out-of-time' (например, my_date_train_test_split)
    детерминация не требуется, поскольку данные разделяются по времени,
    а не случайным образом.
""";

### 4. **Implement the next cross-validation methods:**

#### a. K-Fold, where k is the input parameter, returns a list of train and test indices.

In [10]:
# 4.a
def my_kfold_cv(X, k=5):
    """K-Fold разбиение: возвращает список (train_idx, test_idx)."""
    if X is None or len(X) == 0:
        raise ValueError("X не должен быть пустым")
    if k <= 1 or k > len(X):
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    result = []
    n = len(X)
    all_idx = np.arange(n)
    folds = np.array_split(all_idx, k)

    for test_idx in folds:
        train_idx = all_idx[~np.isin(all_idx, test_idx)]
        result.append((train_idx, test_idx))

    return result

#### b. Grouped K-Fold, where k and group_field are input parameters, returns list of train and test indices.

In [11]:
# 4.b
def my_group_kfold_cv(X, group_field, k=5):
    """Group K-Fold: разбиение по группам без их пересечений."""
    if X is None or len(X) == 0:
        raise ValueError("X не должен быть пустым")
    if group_field is None:
        raise ValueError("'group_field' должен быть задан")
    if group_field not in X.columns:
        raise ValueError(f"Поле '{group_field}' отсутствует в X")
    if k <= 1 or k > len(X):
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    result = []
    n = len(X)
    group_values = np.asarray(X[group_field])

    unique_groups, inv_idx, counts = np.unique(
        group_values, return_inverse=True, return_counts=True
    )
    if len(unique_groups) < k:
        raise ValueError("Уникальных групп меньше, чем k")

    indices_by_group = {}
    for i, g in enumerate(unique_groups):
        indices_by_group[g] = np.where(inv_idx == i)[0].tolist()

    order = np.argsort(counts)[::-1]
    unique_groups = unique_groups[order]
    counts = counts[order]

    fold_sizes = np.zeros(k, dtype=int)
    folds = [[] for _ in range(k)]
    for g, c in zip(unique_groups, counts):
        fold_id = np.argmin(fold_sizes)
        folds[fold_id].extend(indices_by_group[g])
        fold_sizes[fold_id] += c

    for test_idx in folds:
        mask = np.ones(n, dtype=bool)
        mask[test_idx] = False
        train_idx = np.nonzero(mask)[0]
        result.append((train_idx, np.array(sorted(test_idx))))

    return result

#### c. Stratified K-fold, where k and stratify_field are input parameters, returns list of train and test indices.

In [12]:
df["price_category"] = pd.cut(
    df["price"],
    bins=[-np.inf, 2700, 3800, np.inf],
    labels=["low", "middle", "high"]
)
df["price_category"].value_counts()

price_category
low       17286
middle    17269
high      14797
Name: count, dtype: int64

In [13]:
# 4.c
def my_stratified_kfold_cv(X, stratify_field, k=5):
    """Stratified K-Fold: сохраняет пропорции классов в каждом фолде."""
    if X is None or len(X) == 0:
        raise ValueError("X не должен быть пустым")
    if stratify_field is None:
        raise ValueError("'stratify_field' должен быть задан")
    if stratify_field not in X.columns:
        raise ValueError(f"Поле '{stratify_field}' отсутствует в X")
    if k <= 1 or k > len(X):
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    result = []
    n = len(X)
    y = np.asarray(X[stratify_field])

    unique_classes, counts = np.unique(y, return_counts=True)
    if counts.min() < k:
        raise ValueError("Минимальный класс содержит меньше объектов, чем k")

    folds = [[] for _ in range(k)]
    for cls in unique_classes:
        class_idx = np.where(y == cls)[0]
        parts = np.array_split(class_idx, k)
        for i in range(k):
            folds[i].append(parts[i])

    all_idx = np.arange(n)
    for i in range(k):
        test_idx = np.concatenate(folds[i])
        train_idx = np.setdiff1d(all_idx, test_idx, assume_unique=False)
        result.append((train_idx, test_idx))

    return result

#### d. Time series split, where k and date_field are input parameters, returns list of train and test indices.

In [14]:
# 4.d
def my_time_series_split(X, date_field=None, k=5):
    """TimeSeriesSplit: train — прошлое, test — следующий блок."""
    if X is None or len(X) == 0:
        raise ValueError("X не должен быть пустым")
    if date_field is None:
        raise ValueError("'date_field' должен быть задан")
    if date_field not in X.columns:
        raise ValueError(f"Поле '{date_field}' отсутствует в X")
    if k <= 1 or k > len(X):
        raise ValueError("k должен быть в диапазоне [2, len(X)]")

    result = []
    index_array = X.index.to_numpy()
    n = len(index_array)

    n_folds = k + 1
    test_size = n // n_folds
    remainder = n % n_folds
    train_end = remainder + test_size

    for _ in range(k):
        test_start = train_end
        test_end = test_start + test_size
        result.append((index_array[:train_end], index_array[test_start:test_end]))
        train_end += test_size

    return result

### 5. **Cross-validation comparison**

#### a. Apply all the validation methods implemented above to our dataset. To apply Stratified algorithm you should preprocess target.

In [15]:
# 5.a
n_folds = 5

my_train_test_kf = my_kfold_cv(df, k=n_folds)
my_train_test_gkf = my_group_kfold_cv(df, "building_id", k=n_folds)
my_train_test_skf = my_stratified_kfold_cv(df, "price_category", k=n_folds)
my_train_test_tss = my_time_series_split(df, "created", n_folds)

my_methods = {"MyKFold": my_train_test_kf,
              "MyGroupKFold": my_train_test_gkf,
              "MyStratifiedKFold": my_train_test_skf,
              "MyTimeSeriesSplit": my_train_test_tss}

#### b. Apply the appropriate methods from sklearn.

In [16]:
# 5.b
from sklearn.model_selection import (KFold,
                                     GroupKFold,
                                     StratifiedKFold,
                                     TimeSeriesSplit)

train_test_kf = list(KFold(n_folds).split(df))
train_test_gkf = list(GroupKFold(n_folds).split(df, groups=df["building_id"]))
train_test_skf = list(StratifiedKFold(n_folds).split(df, df["price_category"]))
train_test_tss = list(TimeSeriesSplit(n_folds).split(df, "created"))

base_methods = {"KFold": list(train_test_kf),
                "GroupKFold": list(train_test_gkf),
                "StratifiedKFold": list(train_test_skf),
                "TimeSeriesSplit": list(train_test_tss)}

#### c. Compare the resulting feature distributions for the training part of the dataset between sklearn and your implementation.

In [17]:
# 5.c
compare = (lambda x, y: np.array_equal(x, y))
for my, base in zip(my_methods.items(), base_methods.items()):
    for fold in range(n_folds):
        arr1 = base[1][fold][0]
        arr2 = my[1][fold][0]
        print(f"Сравнение {my[0]} и {base[0]}:", ("Идентичны"
                                                  if compare(arr1, arr2)
                                                  else "Различаются"))
    print()

Сравнение MyKFold и KFold: Идентичны
Сравнение MyKFold и KFold: Идентичны
Сравнение MyKFold и KFold: Идентичны
Сравнение MyKFold и KFold: Идентичны
Сравнение MyKFold и KFold: Идентичны

Сравнение MyGroupKFold и GroupKFold: Идентичны
Сравнение MyGroupKFold и GroupKFold: Идентичны
Сравнение MyGroupKFold и GroupKFold: Идентичны
Сравнение MyGroupKFold и GroupKFold: Идентичны
Сравнение MyGroupKFold и GroupKFold: Идентичны

Сравнение MyStratifiedKFold и StratifiedKFold: Различаются
Сравнение MyStratifiedKFold и StratifiedKFold: Различаются
Сравнение MyStratifiedKFold и StratifiedKFold: Различаются
Сравнение MyStratifiedKFold и StratifiedKFold: Различаются
Сравнение MyStratifiedKFold и StratifiedKFold: Различаются

Сравнение MyTimeSeriesSplit и TimeSeriesSplit: Идентичны
Сравнение MyTimeSeriesSplit и TimeSeriesSplit: Идентичны
Сравнение MyTimeSeriesSplit и TimeSeriesSplit: Идентичны
Сравнение MyTimeSeriesSplit и TimeSeriesSplit: Идентичны
Сравнение MyTimeSeriesSplit и TimeSeriesSplit: Идентич

In [18]:
# 5.c.a
display(
    my_train_test_skf,
    train_test_skf
)

[(array([ 9785,  9786,  9787, ..., 49349, 49350, 49351], shape=(39480,)),
  array([   5,    6,    9, ..., 9819, 9821, 9823], shape=(9872,))),
 (array([    0,     1,     2, ..., 49349, 49350, 49351], shape=(39481,)),
  array([ 9785,  9786,  9787, ..., 19650, 19651, 19652], shape=(9871,))),
 (array([    0,     1,     2, ..., 49349, 49350, 49351], shape=(39482,)),
  array([19535, 19537, 19541, ..., 29693, 29696, 29698], shape=(9870,))),
 (array([    0,     1,     2, ..., 49349, 49350, 49351], shape=(39482,)),
  array([29197, 29198, 29199, ..., 39555, 39557, 39560], shape=(9870,))),
 (array([    0,     1,     2, ..., 39721, 39723, 39727], shape=(39483,)),
  array([39090, 39111, 39112, ..., 49344, 49345, 49347], shape=(9869,)))]

[(array([ 9785,  9786,  9787, ..., 49349, 49350, 49351], shape=(39481,)),
  array([    0,     1,     2, ..., 10024, 10030, 10035], shape=(9871,))),
 (array([    0,     1,     2, ..., 49349, 49350, 49351], shape=(39481,)),
  array([ 9785,  9786,  9787, ..., 19978, 19981, 19983], shape=(9871,))),
 (array([    0,     1,     2, ..., 49349, 49350, 49351], shape=(39482,)),
  array([19535, 19537, 19541, ..., 29872, 29874, 29876], shape=(9870,))),
 (array([    0,     1,     2, ..., 49349, 49350, 49351], shape=(39482,)),
  array([29197, 29198, 29199, ..., 39721, 39723, 39727], shape=(9870,))),
 (array([    0,     1,     2, ..., 39721, 39723, 39727], shape=(39482,)),
  array([39090, 39111, 39112, ..., 49349, 49350, 49351], shape=(9870,)))]

#### d. Compare all validation schemes. Choose the best one. Explain your choice.

##### __5.d__
> Изучив "под капотом" каждый метод кросс валидации, я пришел к выводу, что лучшим методом является `StratifiedKFold` для стратегии `out-of-fold`, т.к. метод `StratifiedKFold` сохраняет баланс в целевых признаках, что уменьшает смещение предсказаний в сторону одной метки. Например, в нашем случае `df["price"]` имеет достаточно высокие __*стандартное отклонение $\left(\sigma ^ 2\right)$ и дисперсию $\left(\sigma = \sqrt{\sigma ^ 2}\right)$*__, что говорит о высоком разбросе целевого признака `"price"` в датафрэйме. Также, визуализация распределения `"price"` из предыдущих задач показала, что `"price"` имеет сильное левостороннее смещение (длинный правый хвост, распределение не является нормальным), а это значит, что модель чаще будет предсказывать значения области "высокой плотности". Мы можем обработать это смещение, либо преобразовать его из чисел в __*диапазоны/категории*__. `StratifiedKFold` требует предобработки числовых меток и это следует учитывать.

### 6. **Feature Selection**

#### a. Fit a Lasso regression model with normalized features. Use your method for splitting samples into 3 parts by field created with 60/20/20 ratio — train/validation/test.

In [19]:
# 6.a
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler


df_prepare = df[rooms + features + target].copy()

scaler = MinMaxScaler()
df_prepare[rooms + features] = scaler.fit_transform(df_prepare[rooms + features])

train, validation, test = my_train_validation_test_split(
    df_prepare.drop(columns="price"),
    df_prepare["price"],
    validation_size=0.2,
    test_size=0.2,
    random_state=21,
)
lasso = Lasso(alpha=32.3809, random_state=21, max_iter=10_000)
lasso.fit(train.drop(columns="price"), train["price"])

,alpha,32.3809
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,10000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,21
,selection,'cyclic'


#### b. Sort features by weight coefficients from model, fit model to top 10 features and compare quality.

In [20]:
# 6.b
feature_names = lasso.feature_names_in_
feature_weights = np.abs(lasso.coef_).round(6)
weights = pd.DataFrame({
    "Name": feature_names,
    "Importance": feature_weights}
)
most_important = weights.sort_values(by="Importance", ascending=False)
most_important.reset_index(drop=True)

,Name,Importance
0,bathrooms,7029.967698
1,bedrooms,5719.554972
2,Doorman,1285.859918
3,LaundryinUnit,488.325535
4,Elevator,432.521756
5,LaundryinBuilding,423.666310
6,NoFee,279.017578
7,HardwoodFloors,247.807192
8,DogsAllowed,240.873815
9,LaundryInBuilding,137.935836


#### c. Implement method for simple feature selection by nan-ratio in feature and correlation. Apply this method to feature set and take top 10 features, refit model and measure quality.

#### d. Implement permutation importance method and take top 10 features, refit model and measure quality.

#### e. Import Shap and also refit model on top 10 features.

#### f. Compare the quality of these methods for different aspects — speed, metrics and stability.

### 7. **Hyperparameter optimization**

#### a. Implement grid search and random search methods for alpha and l1_ratio for sklearn's ElasticNet model.

#### b. Find the best combination of model hyperparameters.

#### c. Fit the resulting model.

#### d. Import optuna and configure the same experiment with ElasticNet.

#### e. Estimate metrics and compare approaches.

#### f. Run optuna on one of the cross-validation schemes.